In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

# Go up one directory from the current working dir (notebooks → project root)
project_root = Path(os.getcwd()).resolve().parent


In [ ]:
from datetime import date
from scripts.knmi_pull import fetch_knmi_prec_evap
import os
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go



station = 249  # Berkhout
start_date = "20220101"
end_date = date.today().strftime("%Y%m%d")

prec, evap = fetch_knmi_prec_evap(station, start_date, end_date)

print(f"KNMI data opgehaald voor station {station} van {start_date} tot {end_date}.")
print(f"Aantal dagen neerslag: {len(prec)}")
print(f"Aantal dagen verdamping: {len(evap)}")

# Toon de eerste paar regels
print(prec.head())
print(evap.head())

In [ ]:
# Define input and output directories relative to the project
input_dir  = project_root / 'output_files' / 'output_sheets'
output_dir = project_root / 'output_files' / 'output_graphs_median'

# Make sure the output folder exists
output_dir.mkdir(parents=True, exist_ok=True)

# Pre-fetch KNMI data
knmi_full = pd.DataFrame({
    'KNMI_Precipitation': prec,
    'KNMI_Evapotranspiration': evap
})
knmi_full.index.name = 'Timestamp'
knmi_full = knmi_full.reset_index()
knmi_full['Timestamp'] = pd.to_datetime(knmi_full['Timestamp'])
knmi_full['KNMI_Recharge'] = (
    knmi_full['KNMI_Precipitation'] - knmi_full['KNMI_Evapotranspiration']
)

# Loop over every CSV in the input folder
for local_csv in input_dir.glob('*.csv'):
    df_local = pd.read_csv(local_csv, parse_dates=['Timestamp'])

    # Identify relevant columns
    y2_col   = next((c for c in df_local.columns if 'head' in c.lower()), None)
    prec_col = next((c for c in df_local.columns if 'neerslag' in c.lower() or 'precip' in c.lower()), None)
    evap_col = next((c for c in df_local.columns if 'verdamp' in c.lower() or 'evap' in c.lower()), None)

    # Resample water level to daily median
    if y2_col:
        df_local[y2_col] = pd.to_numeric(df_local[y2_col], errors='coerce')
        df_y2_median = (
            df_local.set_index('Timestamp')[[y2_col]]
            .resample('D')
            .median()
            .reset_index()
        )
    else:
        df_y2_median = None

    # Merge full local with KNMI
    merged = pd.merge(
        df_local,
        knmi_full,
        on='Timestamp',
        how='left'
    )

    # Determine plot range
    crop_start = df_local['Timestamp'].min()
    crop_end   = df_local['Timestamp'].max()

    # Build figure
    fig = go.Figure()
    # KNMI traces (full date index, but view-cropped)
    fig.add_trace(go.Scatter(
        x=knmi_full['Timestamp'], y=knmi_full['KNMI_Precipitation'],
        name='KNMI Precipitation', yaxis='y1', line=dict(width=1.5)
    ))
    fig.add_trace(go.Scatter(
        x=knmi_full['Timestamp'], y=knmi_full['KNMI_Evapotranspiration'],
        name='KNMI Evapotranspiration', yaxis='y1', line=dict(width=1.5)
    ))
    fig.add_trace(go.Scatter(
        x=knmi_full['Timestamp'], y=knmi_full['KNMI_Recharge'],
        name='KNMI Recharge', yaxis='y1', line=dict(width=1.5)
    ))
    # Local traces
    if prec_col:
        fig.add_trace(go.Scatter(
            x=merged['Timestamp'], y=merged[prec_col],
            name='Local Precipitation', yaxis='y1', line=dict(width=2)
        ))
    if evap_col:
        fig.add_trace(go.Scatter(
            x=merged['Timestamp'], y=merged[evap_col],
            name='Local Evaporation', yaxis='y1', line=dict(width=2)
        ))
    if df_y2_median is not None:
        fig.add_trace(go.Scatter(
            x=df_y2_median['Timestamp'], y=df_y2_median[y2_col],
            name='head (Daily Median)', yaxis='y2', line=dict(width=2)
        ))

    # Layout
    fig.update_layout(
        title=f'KNMI vs Local Series (Daily Median head): {local_csv.stem}',
        xaxis_title='Datum',
        yaxis=dict(title='mm/dag (KNMI & Local)', side='left'),
        yaxis2=dict(
            title='head', overlaying='y',
            side='right', showgrid=False
        ),
        legend_title='Variabele',
        hovermode='x unified',
        template='plotly_white',
        width=1200,
        height=600,
        xaxis=dict(range=[crop_start, crop_end])
    )

    # Save the plot
    out_html = output_dir / f'{local_csv.stem}.html'
    fig.write_html(out_html)
    print(f'✅ Plot with daily median saved to: {out_html}')